<a href="https://colab.research.google.com/github/ab0473a/Machine-Learning-/blob/deeplearning.ai/Happy_or_Sad_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf 
import os 
import zipfile

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

DESIRED_ACCURACY = 0.999

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if (logs.get('acc') > DESIRED_ACCURACY):
      print('\nDesired Accuracy is met, Stopping training...')
      self.model.stop_training = True

callbacks = myCallback()

--2021-05-25 01:02:40--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 173.194.210.128, 173.194.211.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2021-05-25 01:02:40 (132 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [11]:
#This code block defines and compiles the training model
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)), #1st Conv layer to apply 3x3 filter upon input shape of 150x150 color pic
  tf.keras.layers.MaxPooling2D(2,2), #Max pooling will take the features in the picture and reduce the overall px by 1/2
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'), #2nd Conv layer with same 64 neurons which will again apply the 3x3 filter utilizing a relu optimizer which activates as 1 if data >0
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'), #This is the hidden layer of 128 neurons doing the actual work once the conv layers have processed all the data 
  tf.keras.layers.Dense(1, activation='sigmoid') #Using 1 output channel because this is a binary classifier of 'happy' or 'sad'
])

from tensorflow.keras.optimizers import RMSprop 

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255.0)

train_generator = train_datagen.flow_from_directory(
    '/tmp/h-or-s/',
    target_size=(150,150),
    batch_size=80,
    class_mode='binary')

Found 80 images belonging to 2 classes.


In [16]:
#This code block calls model.fit and does the training for a number of epochs until the accuracy is reached 
history = model.fit_generator(
    train_generator,
    steps_per_epoch=1,
    epochs=30,
    verbose=1,
    callbacks=[callbacks]
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
1/1 [==============================] - 6s 6s/step - loss: 0.6940 - acc: 0.5000
Epoch 2/30
1/1 [==============================] - 4s 4s/step - loss: 0.7619 - acc: 0.5000
Epoch 3/30
1/1 [==============================] - 4s 4s/step - loss: 0.7119 - acc: 0.5000
Epoch 4/30
1/1 [==============================] - 4s 4s/step - loss: 0.6708 - acc: 0.5000
Epoch 5/30
1/1 [==============================] - 4s 4s/step - loss: 0.5985 - acc: 0.5000
Epoch 6/30
1/1 [==============================] - 4s 4s/step - loss: 0.5121 - acc: 0.7500
Epoch 7/30
1/1 [==============================] - 4s 4s/step - loss: 0.4012 - acc: 0.9250
Epoch 8/30
1/1 [==============================] - 4s 4s/step - loss: 0.2871 - acc: 0.9375
Epoch 9/30
1/1 [==============================] - 4s 4s/step - loss: 0.2135 - acc: 0.9500
Epoch 10/30
1/1 [==============================] - 5s 5s/step - loss: 0.3016 - acc: 0.9000
Epoch 11/30
1/1 [==============================] - 6s 6s/step - loss: 0.2900 - acc: 0.8000
Epoch 12

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 34, 34, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 18496)            

In [24]:
classifications = model.predict(train_generator)
print(classifications[0])


[0.98834866]
